<a href="https://colab.research.google.com/github/agarwalpratik/aiml/blob/main/FeatureEngg_Melbourne_Housing_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

mb = pd.read_csv('melb_data.csv')
mb = mb[['Car', 'Landsize', 'BuildingArea', 'YearBuilt','Price']]
mb.head()

,Car,Landsize,BuildingArea,YearBuilt,Price
0,1.0,202.0,NaN,NaN,1480000.0
1,0.0,156.0,79.0,1900.0,1035000.0
2,0.0,134.0,150.0,1900.0,1465000.0
3,1.0,94.0,NaN,NaN,850000.0
4,2.0,120.0,142.0,2014.0,1600000.0


In [ ]:
mb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Car           13518 non-null  float64
 1   Landsize      13580 non-null  float64
 2   BuildingArea  7130 non-null   float64
 3   YearBuilt     8205 non-null   float64
 4   Price         13580 non-null  float64
dtypes: float64(5)
memory usage: 530.6 KB


In [ ]:
mb['Landsize'].isna().sum(), mb['Car'].isna().sum(), mb['BuildingArea'].isna().sum(), mb['YearBuilt'].isna().sum()

(0, 62, 6450, 5375)

In [ ]:
#Statistical approach
mb_stat = mb

# Considering Car parking & YearBuilt as discrete numeric and could be defaulted with median()
mb_stat['Car'].fillna(mb_stat['Car'].median(), inplace=True)
mb_stat['YearBuilt'].fillna(mb_stat['YearBuilt'].median(), inplace=True)

# Considering Landsize & BuildingArea as continuous numeric and could be defaulted with mean()
#mb_stat['Landsize'].fillna(mb_stat['Landsize'].mean(), inplace=True)
mb_stat['BuildingArea'].fillna(mb_stat['BuildingArea'].mean(), inplace=True)
mb_stat.describe()

,Car,Landsize,BuildingArea,YearBuilt,Price
count,13580.000000,13580.000000,13580.000000,13580.000000,1.358000e+04
mean,1.611856,558.416127,151.967650,1966.788218,1.075684e+06
std,0.960793,3990.669241,392.002962,29.088642,6.393107e+05
min,0.000000,0.000000,0.000000,1196.000000,8.500000e+04
25%,1.000000,177.000000,122.000000,1960.000000,6.500000e+05
50%,2.000000,440.000000,151.967650,1970.000000,9.030000e+05
75%,2.000000,651.000000,151.967650,1975.000000,1.330000e+06
max,10.000000,433014.000000,44515.000000,2018.000000,9.000000e+06


In [ ]:
#Domain approach
mb_domain = mb
mb_domain['Car'].fillna(1, inplace=True)
mb_domain['Car'].replace(0,1,inplace=True)
mb_domain['BuildingArea'].fillna(80, inplace=True)
mb_domain['BuildingArea'].replace(0,80, inplace=True)
mb_domain['YearBuilt'].fillna(2000, inplace=True)
mb_domain.describe()


,Car,Landsize,BuildingArea,YearBuilt,Price
count,13580.000000,13580.000000,13580.000000,13580.000000,1.358000e+04
mean,1.687408,558.416127,152.067797,1966.788218,1.075684e+06
std,0.865677,3990.669241,391.974341,29.088642,6.393107e+05
min,1.000000,0.000000,1.000000,1196.000000,8.500000e+04
25%,1.000000,177.000000,122.000000,1960.000000,6.500000e+05
50%,2.000000,440.000000,151.967650,1970.000000,9.030000e+05
75%,2.000000,651.000000,151.967650,1975.000000,1.330000e+06
max,10.000000,433014.000000,44515.000000,2018.000000,9.000000e+06


In [ ]:
#Statistical approach
features = mb_stat.drop('Price', axis=1).values
label = mb_stat['Price'].values

In [ ]:
#Domain approach
features = mb_domain.drop('Price', axis=1).values
label = mb_domain['Price'].values

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

def getBestModel(technique,features,label,CL=0.95,test_size=0.2):
  approvedModels = []
  for rs in range(1,100):
    xTrain,xTest,yTrain,yTest = train_test_split(features,label,test_size=0.2,random_state=rs)

    model = LinearRegression()
    model.fit(xTrain,yTrain)
    trainScore = model.score(xTrain,yTrain)
    testScore = model.score(xTest,yTest)

    if testScore > trainScore and testScore >= CL:
      approvedModels.append([testScore,trainScore,rs,model])
      #print(f"Model APPROVED, Test score: {testScore}, Train score: {trainScore} & Random State: {rs}")

  if len(approvedModels) > 0:
    approvedModels.sort(reverse=True)

    bestModel = approvedModels[0][3]
    print(f"Best model[{technique}]: Test score: {approvedModels[0][0]}, Train score: {approvedModels[0][1]} & Random State: {approvedModels[0][2]}")
    return bestModel
  else:
    print("No model approved")
    return None


In [ ]:
bestModel_lr = getBestModel('Linear Regression',features,label,0.10,0.2)


Best model[Linear Regression]: Test score: 0.17938530230102467, Train score: 0.13679385743379702 & Random State: 57


# 2) Correlation Analysis Method

In [ ]:
mb_domain.corr()

,Car,Landsize,BuildingArea,YearBuilt,Price
Car,1.000000,0.024783,0.072387,0.024016,0.265846
Landsize,0.024783,1.000000,0.094663,0.008806,0.037507
BuildingArea,0.072387,0.094663,1.000000,0.005255,0.069724
YearBuilt,0.024016,0.008806,0.005255,1.000000,-0.259387
Price,0.265846,0.037507,0.069724,-0.259387,1.000000


In [ ]:
#Guideline:
# Select feature columns that have corr greater than or equal to 50% w.r.t label column
# Therefore selected features: RDspend and MarkSpend

#No columns have corr() of more than 50% (0.5), hence Correlation Analysis does not help with feature selection for this dataset.

# Conclusion: Linear Model remains unchanged!

# 3) OLS Method

In [ ]:
#Step1: Perform all in (Adding intercept feature column in existing feature array)

featuresAllIn = np.concatenate((np.ones((len(features),1)),features),axis=1)
featuresAllIn


array([[1.0000000e+00, 1.0000000e+00, 2.0200000e+02, 1.5196765e+02,
        1.9700000e+03],
       [1.0000000e+00, 1.0000000e+00, 1.5600000e+02, 7.9000000e+01,
        1.9000000e+03],
       [1.0000000e+00, 1.0000000e+00, 1.3400000e+02, 1.5000000e+02,
        1.9000000e+03],
       ...,
       [1.0000000e+00, 4.0000000e+00, 4.3600000e+02, 1.5196765e+02,
        1.9970000e+03],
       [1.0000000e+00, 5.0000000e+00, 8.6600000e+02, 1.5700000e+02,
        1.9200000e+03],
       [1.0000000e+00, 1.0000000e+00, 3.6200000e+02, 1.1200000e+02,
        1.9200000e+03]])

In [ ]:
#Step2: Decide SL
SL = 0.05

In [ ]:
#Step3: Perform OLS
# OLS
# endog -- label column
# exog --- feature column

import statsmodels.regression.linear_model as stat

ols_model = stat.OLS(endog=label, exog=featuresAllIn).fit()
ols_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.145
Model:                            OLS   Adj. R-squared:                  0.145
Method:                 Least Squares   F-statistic:                     574.7
Date:                Sun, 15 Sep 2024   Prob (F-statistic):               0.00
Time:                        05:32:18   Log-Likelihood:            -1.9975e+05
No. Observations:               13580   AIC:                         3.995e+05
Df Residuals:                   13575   BIC:                         3.995e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.224e+07   3.43e+05     35.669      0.000    1.16e+07    1.29e+07
x1          1.979e+05   5879.609     33.660      0.000    1.86e+05    2.09e+05
x2             4.5771      1.278      3.583      0.000       2.073       7.081
x3            79.9527     13.036      6.133      0.000      54.400     105.505
x4         -5853.4579    174.498    -33.545      0.000   -6195.498   -5511.418
==============================================================================
Omnibus:                     6846.247   Durbin-Watson:                   1.466
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            79079.501
Skew:                           2.151   Prob(JB):                         0.00
Kurtosis:                      14.011   Cond. No.                     2.73e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.73e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
#Step4: Select the feature that has the highest p value

ols_model.pvalues[ols_model.pvalues > 0.5]

array([], dtype=float64)

In [ ]:
#Step5: if pvalue > SL : eliminate that feature and recreate new one
# else: stop

#Conclusion: None of the columns can be eliminated so the model remains unchanged.

# 4) RFE: Recursive Feature Elimination

In [ ]:
#Step 1: Select a linear regression model as the estimator model and create RFE model
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

rfe_estimator_model = LinearRegression()
rfe = RFE(estimator=rfe_estimator_model)

#Step 2: Fit the fearure
rfe.fit(features,label)


RFE(estimator=LinearRegression())

In [ ]:
rfe.ranking_

array([1, 3, 2, 1])

In [ ]:
#Step 3: Preserve features with ranking 1 & 2, eliminate others
# features_rfe = features[:,[0,2,3]]

features_rfe = rfe.transform(features)


In [ ]:
features_rfe

array([[1.000e+00, 1.970e+03],
       [1.000e+00, 1.900e+03],
       [1.000e+00, 1.900e+03],
       ...,
       [4.000e+00, 1.997e+03],
       [5.000e+00, 1.920e+03],
       [1.000e+00, 1.920e+03]])

In [ ]:
#Step 4: Create model using RFE feature set
bestModel_rfe = getBestModel('RFE',features_rfe,label,0.10,0.2)


Best model[RFE]: Test score: 0.17436419367023448, Train score: 0.13379291201880505 & Random State: 57


# 5) SME: Select From Model

In [ ]:
#Step 1: Select a linear regression model as the estimator model and create RFE model
from sklearn.feature_selection import SelectFromModel
sfm_estimator_model = LinearRegression()
sfm = SelectFromModel(estimator=sfm_estimator_model)

#Step 2: Fit the fearure
sfm.fit(features,label)

SelectFromModel(estimator=LinearRegression())

In [ ]:
# Step 3: Get Support
sfm.get_support()

array([ True, False, False, False])

In [ ]:
features_sfm = sfm.transform(features)
# label_sfm = label

features_sfm

array([[1.],
       [1.],
       [1.],
       ...,
       [4.],
       [5.],
       [1.]])

In [ ]:
# Step 4: Get best model based on SFM features
bestModel_sfm = getBestModel('SFM',features_sfm,label,0.0,0.2)

Best model[SFM]: Test score: 0.09523742022768156, Train score: 0.06460424744071447 & Random State: 42


In [ ]:
#Summary

bestModel_lr = getBestModel('LR',features,label,0.0,0.2)
bestModel_rfe = getBestModel('RFE',features_rfe,label,0.0,0.2)
bestModel_sfm = getBestModel('SFM',features_sfm,label,0.0,0.2)

Best model[LR]: Test score: 0.17938530230102467, Train score: 0.13679385743379702 & Random State: 57
Best model[RFE]: Test score: 0.17436419367023448, Train score: 0.13379291201880505 & Random State: 57
Best model[SFM]: Test score: 0.09523742022768156, Train score: 0.06460424744071447 & Random State: 42


In [ ]:
#Export model
# import pickle
# pickle.dump(bestModel,open('melbourneHousingPricePredictor.mdl','wb'))